# TPU approach

I'm learning to use TPUs (see https://www.kaggle.com/code/ryanholbrook/getting-started-with-tpus), so this is my attempt for this competition.

**Note** TPU is not currently working, I've asked in the forums https://www.kaggle.com/discussions/questions-and-answers/582296

In [1]:
import math, re, os
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
#from kaggle_datasets import KaggleDatasets
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

import pandas as pd
#import polars as pl
from sklearn.preprocessing import LabelEncoder
#from kaggle_evaluation.api import GesturePredictor

E0000 00:00:1748597063.035843      10 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:230


In [2]:
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None
    print("Not running on TPU")

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Tensorflow version 2.18.0
Not running on TPU
REPLICAS:  1


In [3]:
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


## Read Data

In [3]:
# Load data
train_df = pd.read_csv("/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv")
targets = train_df[['sequence_id', 'gesture']].drop_duplicates()


In [4]:
train_df.head()

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,tof_5_v54,tof_5_v55,tof_5_v56,tof_5_v57,tof_5_v58,tof_5_v59,tof_5_v60,tof_5_v61,tof_5_v62,tof_5_v63
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,-1.0,-1.0,112.0,119.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,-1.0,-1.0,101.0,111.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,-1.0,-1.0,101.0,109.0,125.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [5]:
train_demographics = pd.read_csv("/kaggle/input/cmi-detect-behavior-with-sensor-data/train_demographics.csv")
train_demographics.head()

,subject,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm
0,SUBJ_000206,1,41,1,1,172.0,50,25.0
1,SUBJ_001430,0,11,0,1,167.0,51,27.0
2,SUBJ_002923,1,28,1,0,164.0,54,26.0
3,SUBJ_003328,1,33,1,1,171.0,52,25.0
4,SUBJ_004117,0,15,0,1,184.0,54,28.0


### Merge all in one dataset

There may be smarter way to do this merge, but for now, let's put it all together.

In [6]:
train_df = train_df.merge(train_demographics, on='subject', how='left')
train_df.fillna(method='ffill', inplace=True)

/tmp/ipykernel_10/234058094.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_df.fillna(method='ffill', inplace=True)


In [7]:
train_df.head()

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,tof_5_v61,tof_5_v62,tof_5_v63,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,-1.0,-1.0,-1.0,0,12,1,1,163.0,52,24.0
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,-1.0,-1.0,-1.0,0,12,1,1,163.0,52,24.0
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,-1.0,-1.0,-1.0,0,12,1,1,163.0,52,24.0
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,-1.0,-1.0,-1.0,0,12,1,1,163.0,52,24.0
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,-1.0,-1.0,-1.0,0,12,1,1,163.0,52,24.0


In [8]:
train_df.describe()


,sequence_counter,acc_x,acc_y,acc_z,rot_w,rot_x,rot_y,rot_z,thm_1,thm_2,...,tof_5_v61,tof_5_v62,tof_5_v63,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm
count,574945.000000,574945.000000,574945.000000,574945.000000,574945.000000,574945.000000,574945.000000,574945.000000,574945.000000,574945.000000,...,574945.000000,574945.000000,574945.000000,574945.000000,574945.000000,574945.000000,574945.000000,574945.000000,574945.000000,574945.000000
mean,43.645234,1.639980,1.790704,-0.459811,0.359589,-0.119262,-0.059807,-0.187373,27.086830,27.142138,...,31.747475,28.780502,27.129085,0.504544,21.630276,0.600247,0.875898,167.647967,51.502566,25.418165
std,45.717673,5.781259,5.003945,6.096490,0.225530,0.466052,0.543588,0.504215,3.226488,2.943012,...,60.412848,57.104146,55.324165,0.499980,10.343351,0.489848,0.329699,10.740996,4.900248,2.964853
min,0.000000,-34.585938,-24.402344,-42.855469,0.000000,-0.999146,-0.999695,-0.998169,-0.370413,21.958820,...,-1.000000,-1.000000,-1.000000,0.000000,10.000000,0.000000,0.000000,135.000000,41.000000,18.000000
25%,17.000000,-2.964844,-2.121094,-5.417969,0.179443,-0.456421,-0.512878,-0.626587,24.764328,24.545063,...,-1.000000,-1.000000,-1.000000,0.000000,13.000000,0.000000,1.000000,163.000000,49.000000,24.000000
50%,35.000000,2.972656,0.695312,-1.562500,0.339417,-0.186523,-0.112671,-0.263000,26.992983,26.372349,...,-1.000000,-1.000000,-1.000000,1.000000,20.000000,1.000000,1.000000,170.000000,52.000000,25.000000
75%,54.000000,6.343750,6.816406,5.164062,0.502502,0.205933,0.442993,0.252930,29.431932,29.631985,...,41.000000,34.000000,29.000000,1.000000,27.000000,1.000000,1.000000,174.000000,55.000000,27.000000
max,699.000000,46.328125,27.183594,30.078125,0.999390,0.999817,0.999451,0.999878,38.457664,37.578339,...,249.000000,249.000000,249.000000,1.000000,53.000000,1.000000,1.000000,190.500000,71.000000,44.000000


In [9]:
', '.join(train_df.columns.to_list())

'row_id, sequence_type, sequence_id, sequence_counter, subject, orientation, behavior, phase, gesture, acc_x, acc_y, acc_z, rot_w, rot_x, rot_y, rot_z, thm_1, thm_2, thm_3, thm_4, thm_5, tof_1_v0, tof_1_v1, tof_1_v2, tof_1_v3, tof_1_v4, tof_1_v5, tof_1_v6, tof_1_v7, tof_1_v8, tof_1_v9, tof_1_v10, tof_1_v11, tof_1_v12, tof_1_v13, tof_1_v14, tof_1_v15, tof_1_v16, tof_1_v17, tof_1_v18, tof_1_v19, tof_1_v20, tof_1_v21, tof_1_v22, tof_1_v23, tof_1_v24, tof_1_v25, tof_1_v26, tof_1_v27, tof_1_v28, tof_1_v29, tof_1_v30, tof_1_v31, tof_1_v32, tof_1_v33, tof_1_v34, tof_1_v35, tof_1_v36, tof_1_v37, tof_1_v38, tof_1_v39, tof_1_v40, tof_1_v41, tof_1_v42, tof_1_v43, tof_1_v44, tof_1_v45, tof_1_v46, tof_1_v47, tof_1_v48, tof_1_v49, tof_1_v50, tof_1_v51, tof_1_v52, tof_1_v53, tof_1_v54, tof_1_v55, tof_1_v56, tof_1_v57, tof_1_v58, tof_1_v59, tof_1_v60, tof_1_v61, tof_1_v62, tof_1_v63, tof_2_v0, tof_2_v1, tof_2_v2, tof_2_v3, tof_2_v4, tof_2_v5, tof_2_v6, tof_2_v7, tof_2_v8, tof_2_v9, tof_2_v10, tof_2_v1

## Prepare Targets and Feature Selection

We only use a few features for now, to simplify the model.

In [10]:

# Encode gesture labels
label_encoder = LabelEncoder()
targets['gesture_enc'] = label_encoder.fit_transform(targets['gesture'])
gesture2id = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Features to use (IMU only)
FEATURES = [
    'acc_x', 'acc_y', 'acc_z',
    'rot_w', 'rot_x', 'rot_y', 'rot_z'
]



In [11]:

# Feature selection
IMU_FEATURES = ['acc_x', 'acc_y', 'acc_z', 'rot_w', 'rot_x', 'rot_y', 'rot_z']
THERMO_FEATURES = [f'thm_{i}' for i in range(1, 6)]
TOF_FEATURES = [f'tof_{i}_v{j}' for i in range(1, 6) for j in range(64)]
DEMO_FEATURES = ['adult_child', 'age', 'sex', 'handedness', 'height_cm', 'shoulder_to_wrist_cm', 'elbow_to_wrist_cm']

FEATURES = IMU_FEATURES + THERMO_FEATURES + DEMO_FEATURES  # TOF excluded for now due to sparsity

In [12]:
FEATURES

['acc_x',
 'acc_y',
 'acc_z',
 'rot_w',
 'rot_x',
 'rot_y',
 'rot_z',
 'thm_1',
 'thm_2',
 'thm_3',
 'thm_4',
 'thm_5',
 'adult_child',
 'age',
 'sex',
 'handedness',
 'height_cm',
 'shoulder_to_wrist_cm',
 'elbow_to_wrist_cm']

## Prepare Sequences

In [13]:
# Prepare sequences
sequence_ids = train_df['sequence_id'].unique()
X, y = [], []
for seq_id in sequence_ids:
    df = train_df[train_df['sequence_id'] == seq_id]
    if df[FEATURES].isnull().values.any():
        continue  # skip incomplete sequences
    x = df[FEATURES].values.astype(np.float32)
    if x.shape[0] < 64:
        pad_width = 64 - x.shape[0]
        x = np.pad(x, ((0, pad_width), (0, 0)), mode='edge')
    else:
        x = x[:64]
    X.append(x)
    y.append(targets.loc[targets['sequence_id'] == seq_id, 'gesture_enc'].values[0])

X = np.stack(X)
y = np.array(y)

## Build TPU model

Apparently Keras is easier for developing TPU models.

In [14]:
# Build model with TPU strategy
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(64, len(FEATURES))),
        tf.keras.layers.Conv1D(64, 5, padding='same', activation='relu'),
        tf.keras.layers.Conv1D(128, 5, padding='same', activation='relu'),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )



I0000 00:00:1748596222.735026      10 service.cc:148] XLA service 0x571daa876400 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748596222.735068      10 service.cc:156]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1748596222.735072      10 service.cc:156]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1748596222.735075      10 service.cc:156]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1748596222.735078      10 service.cc:156]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1748596222.735081      10 service.cc:156]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1748596222.735084      10 service.cc:156]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1748596222.735086      10 service.cc:156]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1748596222.735089      10 service.cc:156]   StreamExecutor device (7): TPU, 2a886c8


## Train!

In [15]:
# Train
model.fit(X, y, epochs=10, batch_size=64, validation_split=0.1)

Epoch 1/10


NotFoundError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/local/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/local/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/usr/local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/usr/local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 765, in start

  File "/usr/local/lib/python3.10/site-packages/anyio/_core/_eventloop.py", line 74, in run

  File "/usr/local/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 2310, in run

  File "/usr/local/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 217, in run

  File "/usr/local/lib/python3.10/asyncio/base_events.py", line 636, in run_until_complete

  File "/usr/local/lib/python3.10/asyncio/base_events.py", 

## Create Submission Interface

In [ ]:
# Create submission interface
predictor = GesturePredictor()

def preprocess_sequence(df):
    df = df.merge(train_demographics, on='subject', how='left')
    df.fillna(method='ffill', inplace=True)
    x = df[FEATURES].values.astype(np.float32)
    if x.shape[0] < 64:
        pad_width = 64 - x.shape[0]
        x = np.pad(x, ((0, pad_width), (0, 0)), mode='edge')
    else:
        x = x[:64]
    return x

@predictor.predict_function
def predict_fn(df: pd.DataFrame) -> str:
    x = preprocess_sequence(df)
    x = np.expand_dims(x, axis=0)
    probs = model.predict(x, verbose=0)[0]
    pred_idx = np.argmax(probs)
    return label_encoder.inverse_transform([pred_idx])[0]

predictor.save("submission.zip")
